In [34]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

In [35]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [36]:
import pandas as pd
books = pd.read_csv("books.csv")

In [46]:
books

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missing_description,age,words_in_description,title_and_subtitle,tagged_description
0,9780687002825,0687002826,Exclusion and Embrace,"A Theological Exploration of Identity, Otherne...",Miroslav Volf,Religion,http://books.google.com/books/content?id=Cqto7...,Life at the end of the twentieth century prese...,1996.0,4.27,306.0,1895.0,0,28.0,920,Exclusion and Embrace: A Theological Explorati...,9780687002825: Life at the end of the twentiet...
1,9780974320620,NaN,Emotional Intelligence 2.0,NaN,"Travis Bradberry, Jean Greaves, Patrick Lencioni",NaN,https://images.gr-assets.com/books/1328765863m...,"""Emotional Intelligence 2.0 succinctly explain...",2003.0,3.81,895.0,53384.0,0,21.0,681,Emotional Intelligence 2.0,"9780974320620: ""Emotional Intelligence 2.0 suc..."
2,9781904271062,1904271065,King Henry IV Part 2,Second Series,William Shakespeare;A. R. Humphreys,Drama,http://books.google.com/books/content?id=AG4h-...,A. R. Humphreyswas Professor of English at Lei...,1967.0,3.80,336.0,241.0,0,57.0,619,King Henry IV Part 2: Second Series,9781904271062: A. R. Humphreyswas Professor of...
3,9780743223130,NaN,John Adams,NaN,David McCullough,NaN,https://images.gr-assets.com/books/1478144278m...,"The enthralling, often surprising story of Joh...",2001.0,4.05,776.0,215780.0,0,23.0,639,John Adams,"9780743223130: The enthralling, often surprisi..."
4,9780060529960,NaN,The Little House Collection,NaN,"Laura Ingalls Wilder, Garth Williams",NaN,https://s.gr-assets.com/assets/nophoto/book/11...,This nine-book paperback box set of the classi...,1953.0,4.33,753.0,124320.0,0,71.0,664,The Little House Collection,9780060529960: This nine-book paperback box se...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6484,9780310243878,0310243874,Breach of Promise,NaN,James Scott Bell,Fiction,http://books.google.com/books/content?id=RNpBD...,Actor and father Mark Gillen finds out how far...,2004.0,4.00,345.0,358.0,0,20.0,25,Breach of Promise,9780310243878: Actor and father Mark Gillen fi...
6485,9781591162025,1591162025,Inuyasha Ani-Manga,NaN,Rumiko Takahashi,Comics & Graphic Novels,http://books.google.com/books/content?id=mf55s...,A teenage girl travels back in time to medieva...,2004.0,4.36,206.0,5787.0,0,20.0,26,Inuyasha Ani-Manga,9781591162025: A teenage girl travels back in ...
6486,9780440414001,0440414008,"See You Around, Sam!",NaN,Lois Lowry,Juvenile Fiction,http://books.google.com/books/content?id=-oXbx...,"Sam Krupnik, mad at his mother because she won...",1998.0,3.68,128.0,235.0,0,26.0,25,"See You Around, Sam!","9780440414001: Sam Krupnik, mad at his mother ..."
6487,9780618339556,0618339558,Fergus and the Night-Demon,An Irish Ghost Story,Jim Murphy,Juvenile Fiction,http://books.google.com/books/content?id=OKeoo...,"On his way to town to have some fun, a lazy bu...",2006.0,3.71,32.0,70.0,0,18.0,26,Fergus and the Night-Demon: An Irish Ghost Story,9780618339556: On his way to town to have some...


In [37]:
books["tagged_description"]

0       9780687002825: Life at the end of the twentiet...
1       9780974320620: "Emotional Intelligence 2.0 suc...
2       9781904271062: A. R. Humphreyswas Professor of...
3       9780743223130: The enthralling, often surprisi...
4       9780060529960: This nine-book paperback box se...
                              ...                        
6484    9780310243878: Actor and father Mark Gillen fi...
6485    9781591162025: A teenage girl travels back in ...
6486    9780440414001: Sam Krupnik, mad at his mother ...
6487    9780618339556: On his way to town to have some...
6488    9780393307009: A study of the Burgess Shale, a...
Name: tagged_description, Length: 6489, dtype: object

In [38]:
books["tagged_description"].to_csv("tagged_description.txt",sep="\n",index=False,header=False)

In [39]:
raw_documents = TextLoader("tagged_description.txt",encoding="utf-8").load()
Chunk = CharacterTextSplitter(chunk_size=0,chunk_overlap=0,separator="\n")
documents = Chunk.split_documents(raw_documents)


Created a chunk of size 5837, which is longer than the specified 0
Created a chunk of size 4514, which is longer than the specified 0
Created a chunk of size 3939, which is longer than the specified 0
Created a chunk of size 3907, which is longer than the specified 0
Created a chunk of size 3783, which is longer than the specified 0
Created a chunk of size 3646, which is longer than the specified 0
Created a chunk of size 3244, which is longer than the specified 0
Created a chunk of size 3169, which is longer than the specified 0
Created a chunk of size 3031, which is longer than the specified 0
Created a chunk of size 2941, which is longer than the specified 0
Created a chunk of size 2947, which is longer than the specified 0
Created a chunk of size 2878, which is longer than the specified 0
Created a chunk of size 2856, which is longer than the specified 0
Created a chunk of size 2870, which is longer than the specified 0
Created a chunk of size 2844, which is longer than the specifi

In [116]:
print("Number of documents:", len(documents))


Number of documents: 6610


In [54]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='"9780687002825: Life at the end of the twentieth century presents us with a disturbing reality. Otherness, the simple fact of being different in some way, has come to be defined as in and of itself evil. Miroslav Volf contends that if the healing word of the gospel is to be heard today, Christian theology must find ways of speaking that address the hatred of the other. Reaching back to the New Testament metaphor of salvation as reconciliation, Volf proposes the idea of embrace as a theological response to the problem of exclusion. Increasingly we see that exclusion has become the primary sin, skewing our perceptions of reality and causing us to react out of fear and anger to all those who are not within our (ever-narrowing) circle. In light of this, Christians must learn that salvation comes, not only as we are reconciled to God, and not only as we ""learn to live with one another,"" but as we take the dangerous and 

In [55]:
db_books = Chroma.from_documents(documents,embedding=embeddings)

In [66]:
query = "A book about supernatural"
docs = db_books.similarity_search(query,k=5)
docs

[Document(id='3d523be0-8868-4172-a0ab-37d706c1dd0c', metadata={'source': 'tagged_description.txt'}, page_content='"9780099471420: From the author of the extraordinary Vampire Chronicles comes a huge, hypnotic novel of witchcraft and the occult through four centuries.Demonstrating, once again, her gift for spellbinding storytelling and the creation of legend, Anne Rice makes real for us a great dynasty of witches—a family given to poetry and to incest, to murder and to philosophy; a family that, over the ages, is itself haunted by a powerful, dangerous, and seductive being.On the veranda of a great New Orleans house, now faded, a mute and fragile woman sits rocking... and The Witching Hour begins.It begins in our time with a rescue at sea.\xa0\xa0Rowan Mayfair, a beautiful woman, a brilliant practitioner of neurosurgery—aware that she has special powers but unaware that she comes from an ancient line of witches—finds the drowned body of a man off the coast of California and brings him t

In [69]:
raw_isbn = docs[0].page_content.split()[0]
clean_isbn = raw_isbn.strip('":')
isbn_int = int(float(clean_isbn))

books[books["isbn13"] == isbn_int]


,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missing_description,age,words_in_description,title_and_subtitle,tagged_description
33,9780099471420,NaN,The Witching Hour,NaN,Anne Rice,NaN,https://images.gr-assets.com/books/1327289387m...,From the author of the extraordinary Vampire C...,1990.0,4.1,514.0,79824.0,0,34.0,423,The Witching Hour,9780099471420: From the author of the extraord...


In [131]:
def recommendations(query, top_n=10, initial_k=100, max_k=1000, step=100):
    seen_isbns = set()
    results = []

    current_k = initial_k

    while len(results) < top_n and current_k <= max_k:
        recs = db_books.similarity_search(query, k=current_k)

        for doc in recs:
            raw = doc.page_content.split(":")[0]
            isbn_clean = raw.strip('"')

            if isbn_clean in seen_isbns:
                continue

            book = books[books["isbn13"].astype(str) == isbn_clean]
            if not book.empty:
                results.append(book)
                seen_isbns.add(isbn_clean)

            if len(results) >= top_n:
                break

        current_k += step

    if results:
        return pd.concat(results[:top_n], ignore_index=True)
    else:
        return pd.DataFrame()


In [145]:

df = recommendations("supernatural", top_n=20)
print(len(df))


21


In [146]:
df

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missing_description,age,words_in_description,title_and_subtitle,tagged_description
0,9780312624680,NaN,Born at Midnight,NaN,C.C. Hunter,NaN,https://s.gr-assets.com/assets/nophoto/book/11...,Don’t miss this spectacular new series that wi...,2011.0,4.13,251.0,68084.0,0,13.0,210,Born at Midnight,9780312624680: Don’t miss this spectacular new...
1,9780441014740,NaN,Blood Bound,NaN,Patricia Briggs,NaN,https://images.gr-assets.com/books/1358229157m...,"Under the rule of science, there are no witch ...",2007.0,4.31,120.0,84677.0,0,17.0,107,Blood Bound,"9780441014740: Under the rule of science, ther..."
2,9780425209615,042520961X,Shadow of the Moon,NaN,Rebecca York,Fiction,http://books.google.com/books/content?id=9EV0C...,Samantha Carpenter must join forces with repor...,2006.0,3.79,336.0,530.0,0,18.0,46,Shadow of the Moon,9780425209615: Samantha Carpenter must join fo...
3,9780099471420,NaN,The Witching Hour,NaN,Anne Rice,NaN,https://images.gr-assets.com/books/1327289387m...,From the author of the extraordinary Vampire C...,1990.0,4.10,514.0,79824.0,0,34.0,423,The Witching Hour,9780099471420: From the author of the extraord...
4,9780099456452,0099456451,The Spook's Apprentice,NaN,Joseph Delaney,Exorcism,http://books.google.com/books/content?id=ZA0yS...,A wonderful and terrifying series by a new wri...,2005.0,3.98,325.0,2733.0,0,19.0,85,The Spook's Apprentice,9780099456452: A wonderful and terrifying seri...
5,9780061662690,NaN,The Summoning,NaN,Kelley Armstrong,NaN,https://images.gr-assets.com/books/1478986944m...,Chloe Saunders used to have a relatively norma...,2008.0,4.03,193.0,132285.0,0,16.0,177,The Summoning,9780061662690: Chloe Saunders used to have a r...
6,9780515131093,0515131091,Out of this World,NaN,NaN,Fiction,http://books.google.com/books/content?id=Y4-0u...,Collects paranormal and futuristic romantic st...,2001.0,4.07,357.0,5915.0,0,23.0,38,Out of this World,9780515131093: Collects paranormal and futuris...
7,9780441017030,NaN,Magic Strikes,NaN,Ilona Andrews,NaN,https://images.gr-assets.com/books/1221738233m...,Drafted into working for the Order of Merciful...,2009.0,4.44,120.0,51165.0,0,15.0,99,Magic Strikes,9780441017030: Drafted into working for the Or...
8,9780312360310,NaN,Chosen: A House of Night Novel,NaN,"P.C. Cast, Kristin Cast",NaN,https://images.gr-assets.com/books/1393192825m...,Dark forces are at work at the House of Night ...,2008.0,3.90,133.0,180961.0,0,16.0,115,Chosen: A House of Night Novel,9780312360310: Dark forces are at work at the ...
9,9780446359207,0446359203,Carrion Comfort,NaN,Dan Simmons,Fiction,http://books.google.com/books/content?id=37wWp...,"Three elderly friends, who possess supernatura...",1990.0,3.90,884.0,15056.0,0,34.0,33,Carrion Comfort,"9780446359207: Three elderly friends, who poss..."
